In [88]:
import os
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'myzaka.settings')
import django
django.setup()

import urllib3
import requests
import pandas as pd

In [89]:
from app import models

from sqlalchemy import create_engine
engine = create_engine('postgresql://myzaka:myz\@k\@@localhost:5432/myzakadb')

In [62]:
engine.table_names()

['django_migrations',
 'django_content_type',
 'auth_group_permissions',
 'auth_group',
 'auth_user_groups',
 'auth_permission',
 'auth_user_user_permissions',
 'django_admin_log',
 'auth_user',
 'app_audits',
 'app_balancesheet',
 'app_capital',
 'app_cashflow',
 'app_incomeexpense',
 'app_maintenance',
 'app_municipalities',
 'app_wastefulexpenditure',
 'django_session',
 'app_officials']

### Generate Audits data

In [68]:
audits = pd.DataFrame(requests.get('https://municipaldata.treasury.gov.za/api/cubes/audit_opinions/facts',
                                      allow_redirects=True).json()['data'])

In [69]:
audits.head(2)

,demarcation.code,demarcation.label,financial_year_end.year,opinion.code,opinion.label,opinion.report_url
0,BUF,Buffalo City,2011,adverse,Adverse opinion,http://mfma.treasury.gov.za/Documents/07.%20Au...
1,BUF,Buffalo City,2012,qualified,Qualified,http://mfma.treasury.gov.za/Documents/07.%20Au...


In [81]:
audits_df = audits[['demarcation.code', 'demarcation.label', 'financial_year_end.year', 'opinion.label', 'opinion.report_url']]

In [82]:
audits_df.rename(columns={'demarcation.code':'demarcation_code', 'demarcation.label':'demarcation_label', 
                          'financial_year_end.year':'financial_year_end', 'opinion.label':'opinion', 
                          'opinion.report_url':'opinion_report_url'}, inplace=True)

/Users/mabu/anaconda2/envs/ntbhack/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [83]:
audits_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1632 entries, 0 to 1631
Data columns (total 5 columns):
demarcation_code      1632 non-null object
demarcation_label     1632 non-null object
financial_year_end    1632 non-null int64
opinion               1632 non-null object
opinion_report_url    1418 non-null object
dtypes: int64(1), object(4)
memory usage: 63.8+ KB


In [90]:
audits_df.fillna('Not Availiable', inplace=True)
audits_df.to_sql('app_audits', engine, if_exists='append', index=False)
models.Audits.objects.count()

/Users/mabu/anaconda2/envs/ntbhack/lib/python3.6/site-packages/pandas/core/frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


1632

### Create login user

In [9]:
from django.contrib.auth.models import User
User.objects.create_user(
    first_name="Mabu",
    last_name="Manaileng",
    username="mabu",
    email="manailengmj@gmail.com",
    password="@dm1n"
)

### Create Officials

In [15]:
officials = pd.DataFrame(requests.get('https://municipaldata.treasury.gov.za/api/cubes/officials/facts',
                                      allow_redirects=True).json()['data'])

In [18]:
officials.head(2)

,contact_details.email_address,contact_details.fax_number,contact_details.name,contact_details.phone_number,contact_details.title,municipality.demarcation_code,role.role
0,vincentp@buffalocity.gov.za,043 743 9141,Vincent Pillay,043 705 1892,Mr,BUF,Chief Financial Officer
1,zolswam@buffalocity.gov.za,086 545 1288,Zoliswa Patience Matana,043 705 2899,Ms,BUF,Deputy Mayor/Executive Mayor


In [21]:
officials_df = officials[['contact_details.title',  'role.role', 'contact_details.name',  'contact_details.email_address', 
           'contact_details.fax_number', 'contact_details.phone_number', 'municipality.demarcation_code']]

In [46]:
officials_df.rename(columns={'contact_details.title':'title',  'role.role':'role', 'contact_details.name':'name',  
                                                        'contact_details.email_address':'email_address', 'contact_details.fax_number':'fax_number', 
                                                         'contact_details.phone_number':'phone_number', 'municipality.demarcation_code':'demarcation_code'}, inplace=True)

In [65]:
officials_df = officials_df.dropna(subset=['name'])
officials_df.columns

In [50]:
officials_df.fillna('Not Availiable', inplace=True)
officials_df.to_sql('app_officials', engine, if_exists='append', index=False)
models.Officials.objects.count()